<a href="https://colab.research.google.com/github/Aldevand/buzzer-detecction-lstm/blob/main/buzzer_detection_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import library

In [10]:
!pip install transformers
!pip install torch
!pip install tensorflow
!pip install keras
!pip install gensim
!pip install POT # Mengubah nama library dari 'ot' menjadi 'POT'
import pandas as pd
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from datetime import timedelta
from gensim.models import KeyedVectors
from gensim.similarities import WmdSimilarity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 901.7/901.7 kB 13.7 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# Load Dataset

In [2]:
df = pd.read_excel('/content/drive/MyDrive/Keperluan Skripsi/danantara_gabungan.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1255 entries, 0 to 1254
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   conversation_id_str      1255 non-null   int64 
 1   created_at               1255 non-null   object
 2   favorite_count           1255 non-null   int64 
 3   full_text                1255 non-null   object
 4   id_str                   1255 non-null   int64 
 5   image_url                672 non-null    object
 6   in_reply_to_screen_name  112 non-null    object
 7   lang                     1255 non-null   object
 8   location                 810 non-null    object
 9   quote_count              1255 non-null   int64 
 10  reply_count              1255 non-null   int64 
 11  retweet_count            1255 non-null   int64 
 12  tweet_url                1255 non-null   object
 13  user_id_str              1255 non-null   int64 
 14  username                 1255 non-null  

In [4]:
print(df.head(5))
print(df.tail(5))

   conversation_id_str                      created_at  favorite_count  \
0  1900102038295633920  Thu Mar 13 08:30:02 +0000 2025               2   
1  1900122136087585024  Thu Mar 13 09:49:54 +0000 2025               0   
2  1900086851337413120  Thu Mar 13 07:29:41 +0000 2025               0   
3  1898156438125768960  Fri Mar 07 23:38:55 +0000 2025               1   
4  1899762836152155904  Wed Mar 12 10:02:10 +0000 2025               1   

                                           full_text               id_str  \
0  Mengacu dari Indonesia Salary Guide 2025 gaji ...  1900102038295633920   
1  Electrizen Danantara Indonesia telah resmi dib...  1900122136087585024   
2  Wamenkeu Thomas: Danantara Tak Gadai Saham Pem...  1900086851337413120   
3  Proyek DME: Mahal &amp; Merugikan! Gasifikasi ...  1898156438125768960   
4  Masyarakat Indonesia percaya bahwa BPI Dananta...  1899762836152155904   

                                         image_url in_reply_to_screen_name  \
0             

# Text Cleaning

In [5]:
# fungsi pembersihan teks

def cleaning(teks):
  teks = teks.lower()
  teks = re.sub(r'http\S+|www\S+|https\S+', '', teks, flags=re.MULTILINE)
  teks = re.sub(r'@\w+|#\w+', '', teks)
  teks = re.sub(r'[^a-zA-Z\s]', '', teks)
  tokens = word_tokenize(teks)
  list_stopwords = stopwords.words('indonesian')
  filtered_tokens = [token for token in tokens if token not in list_stopwords]
  teks = ' '.join(filtered_tokens)
  return teks
# Terapkan fungsi
df['teks_bersih'] = df['full_text'].apply(cleaning)
print(df[['full_text', 'teks_bersih']].head(5))

                                           full_text  \
0  Mengacu dari Indonesia Salary Guide 2025 gaji ...   
1  Electrizen Danantara Indonesia telah resmi dib...   
2  Wamenkeu Thomas: Danantara Tak Gadai Saham Pem...   
3  Proyek DME: Mahal &amp; Merugikan! Gasifikasi ...   
4  Masyarakat Indonesia percaya bahwa BPI Dananta...   

                                         teks_bersih  
0  mengacu indonesia salary guide gaji selevel ce...  
1  electrizen danantara indonesia resmi dibentuk ...  
2   wamenkeu thomas danantara gadai saham pemerintah  
3  proyek dme mahal amp merugikan gasifikasi batu...  
4  masyarakat indonesia percaya bpi danantara men...  


# Labeling -> Rule Based

Rule Waktu

In [6]:
df['created_at'] = pd.to_datetime(df['created_at'], utc=True)
df.sort_values(by='created_at', inplace=True)
df['label_time'] = 0
time_threshold = timedelta(minutes=5)
for i in range(1, len(df)):
    time_diff = df.iloc[i]['created_at'] - df.iloc[i-1]['created_at']
    if time_diff <= time_threshold:
        df.loc[df.index[i], 'label_time'] = 1
        df.loc[df.index[i-1], 'label_time'] = 1

/tmp/ipython-input-741994207.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['created_at'] = pd.to_datetime(df['created_at'], utc=True)


Rule Kemiripan Teks

In [7]:
# -> unduh GloVe dari lokasi terbaru Menggunakan model GloVe Twitter 27B
!wget https://nlp.stanford.edu/data/glove.twitter.27B.zip -O glove_twitter.zip
!unzip -q glove_twitter.zip

--2025-09-06 17:20:51--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip [following]
--2025-09-06 17:20:52--  https://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520408563 (1.4G) [application/zip]
Saving to: ‘glove_twitter.zip’

glove_twitter.zip   100%[===================>]   1.42G  5.00MB/s    in 4m 45s  

2025-09-06 17:25:38 (5.09 MB/s) - ‘glove_twitter.zip’ saved [1520408563/1520408563]

replace glove.twitter.27B.25d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ena

In [8]:
glove_file = 'glove.twitter.27B.100d.txt'
word_vectors = KeyedVectors.load_word2vec_format(glove_file, binary=False, no_header=True)

print(f"Model Word Embedding Dimuat: {len(word_vectors.index_to_key)} kata, {word_vectors.vector_size} dimensi")

Model Word Embedding Dimuat: 1193514 kata, 100 dimensi


In [14]:
texts_tokenized = [doc.split() for doc in df['teks_bersih'].fillna('')]
df['label_similarity_wmd'] = 0
indices = df.index.to_list()
wmd_threshold = 0.5

print(f"Memulai perhitungan dan pelabelan WMD dengan threshold: {wmd_threshold}")
print(f"Jumlah dokumen: {len(indices)}")
for i in range(len(indices)):
    for j in range(i + 1, len(indices)):
        doc1_tokens = texts_tokenized[i]
        doc2_tokens = texts_tokenized[j]
        if doc1_tokens and doc2_tokens:
            try:
                jarak_wmd = word_vectors.wmdistance(doc1_tokens, doc2_tokens)
                if jarak_wmd < wmd_threshold:
                    df.loc[indices[i], 'label_similarity_wmd'] = 1
                    df.loc[indices[j], 'label_similarity_wmd'] = 1
            except Exception as e:
                print(f"Error saat menghitung WMD antara dokumen {indices[i]} dan {indices[j]}: {e}")
print("\nPelabelan berdasarkan WMD selesai.")
print("Distribusi Label Kemiripan WMD:")
print(df['label_similarity_wmd'].value_counts())
if 1 in df['label_similarity_wmd'].value_counts():
    print("\nContoh Dokumen dengan Label Kemiripan WMD = 1:")
    print(df[df['label_similarity_wmd'] == 1][['full_text', 'teks_bersih', 'label_similarity_wmd']].head())
else:
    print("\nTidak ada dokumen yang diberi label kemiripan WMD = 1.")

Memulai perhitungan dan pelabelan WMD dengan threshold: 0.5
Jumlah dokumen: 1255

Pelabelan berdasarkan WMD selesai.
Distribusi Label Kemiripan WMD:
label_similarity_wmd
1    698
0    557
Name: count, dtype: int64

Contoh Dokumen dengan Label Kemiripan WMD = 1:
                                             full_text  \
640  Erick Thohir menyebutkan sedang mempersiapkan ...   
652  Menteri BUMN Erick Thohir siapkan kantor untuk...   
655  Menteri Badan Usaha Milik Negara (BUMN) Erick ...   
651  Menteri BUMN Erick Thohir mendukung kehadiran ...   
577  Soal Danantara Ini Jawaban Lugas Menteri BUMN ...   

                                           teks_bersih  label_similarity_wmd  
640  erick thohir kantor badan pengelola bp investa...                     1  
652  menteri bumn erick thohir siapkan kantor badan...                     1  
655  menteri badan usaha milik negara bumn erick th...                     1  
651  menteri bumn erick thohir mendukung kehadiran ...                   

-> Gabungkan Rule Waktu dan Similarity/Kemiripan

In [15]:
df['label_final'] = 0
df.loc[(df['label_time'] == 1) | (df['label_similarity_wmd'] == 1), 'label_final'] = 1
print("Pelabelan Final Selesai.")
print("Distribusi label final:")
print(df['label_final'].value_counts())

Pelabelan Final Selesai.
Distribusi label final:
label_final
1    901
0    354
Name: count, dtype: int64
